---
## **主題: 記錄每天攝取的食物，統計當天吃的熱量及營養素**

---
## Import toolkit

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from ipywidgets import interact_manual  #互動套件
import ipywidgets as widgets  #interact控制元件
import requests
from bs4 import BeautifulSoup as bs
from lxml import html
from IPython import display
from matplotlib.font_manager import FontProperties
myf = FontProperties(fname='C:\Windows\Fonts\msjh.ttc')
plt.rcParams['font.sans-serif']=['SimHei']

---
## Function  definition 1 -- search4nutrient()
- 此函式用來找給定食物的熱量及營養素
- Parameter為欲查詢熱量的'食物名稱'


In [2]:
def search4nutrient(name='白飯'):
    # 設定營養資料查詢網頁的rul
    url = 'https://www.cfs.gov.hk/tc_chi/nutrient/search3.php'
    # 送post必需的一些資料
    payload = {
        "keyword":name,
        "keylang": "C",
        "inShortMode": "0",
        "searchcriteria": "and",
        "keyword2": ""
    }
    h = requests.post(url, data=payload)  # 開始送post給網站
    soup = bs(h.text, 'html.parser')  # beautifulsoup解析成html形式
    a = soup.select('.colorTable1 > tr > td > ul > li > a')  # 找到對應食物的href,將使用參數進入食物營養素查詢
    # 取第一個結果:
    if(len(a) > 0):
        a = a[0].attrs['href'].replace('javascript:tosubmit (','')
        a = a.replace(');','').replace("'",'').replace(' ','')
        a = a.split(',')

    # 設定該食物熱量的網址及參數(在網站F12裡找)
    url = 'https://www.cfs.gov.hk/tc_chi/nutrient/fc-myreport.php'
    payload = {
        "fg_id": a[0],
        "fsg_id": a[1],
        "food_id": a[2],
        "inShortMode": ""
    }
    h = requests.post(url, data=payload)  # 開始送post給網站
    soup = bs(h.text, 'html.parser')  # beautifulsoup解析成html形式
    tb = soup.find_all('table')[1]  # 找到第2個table(紀錄熱量的table)
    pf = pd.read_html(str(tb))[0]  # 用read_html直接把該table的html格式轉成dataFrame
    # 以下幾步為處理dataFrame的過程，不多贅述
    pf = pf[(pf['營養']!='主要成分')]
    pf = pf[(pf['營養']!='脂質')]
    pf = pf[(pf['營養']!='礦物質和維他命')]
    pf['數值'].fillna(value=0, inplace=True)
    pf.reset_index(inplace=True, drop=True)
    for i in range(len(pf['營養'])):
        pf['營養'][i] = pf['營養'][i].replace('*','').replace(' ','')
    pf.index = [pf['營養'][i] for i in range(len(pf['營養']))]
    pf.drop('營養',axis=1,inplace=True)
    pf = pf[['數值','單位']]
    pf;
    return pf

search4nutrient()

,數值,單位
能量,130,千卡
蛋白質,2.38,克
碳水化合物,28.59,克
脂肪,0.21,克
膳食纖維,0,克
糖,0,克
飽和脂肪,0.057,克
反式脂肪,0,克
膽固醇,0,毫克
鈣,3,毫克


---
## Function definition 2
- **RecordEat(Date, item, gram, note)**
    - 紀錄吃的東西、重量...

In [3]:
# Date 是屬於datetime.date object, 用Date.strftime('%Y(或%m,%d)')來得到年月日
def RecordEat(Date, item, gram, note):
    # 如果輸入不完整，則不紀錄這筆資料
    if(len(item)==0 or len(gram)==0 or (Date==None)):
        print("請輸入完整內容")
        return
    data = [[Date, item, gram, note]]  # 建list，這裡輸出依序是'日期', '品項', '金額', '備註'
    df = pd.DataFrame(data, columns = ['日期', '食物名稱', '重量(公克)', '備註'])  # 存csv的標準格式:'DataFrame'
    # 如果csv已經存在，則直接寫資料，不寫header
    # 如果不存在，則把header也一起寫進csv檔
    try:
        with open('eatlist.csv','r') as f:
            True
        df.to_csv('eatlist.csv',mode='a', header=False, index=False)
    except:
        df.to_csv('eatlist.csv',mode='w', header=True, index=False)
    print(' '*np.random.randint(10)+'紀錄成功！')
    return

---
## Function definition 3
- **CalculEat(Date)**
    - 秀出特定日期當天攝取的食物熱量、營養素等等

In [4]:
def CalculEat(Date):
    TotalCalo = {'能量':[0,'千卡'], '蛋白質':[0,'克'], '碳水化合物':[0,'克'], '脂肪':[0,'克'], 
                 '膳食纖維':[0,'克'], '糖':[0,'克'], '飽和脂肪':[0,'克'], '反式脂肪':[0,'克'], 
                 '膽固醇':[0,'毫克'], '鈣':[0,'毫克'], '銅':[0,'毫克'], '鐵':[0,'毫克'], 
                 '鎂':[0,'毫克'], '錳':[0,'毫克'], '磷':[0,'毫克'], '鉀':[0,'毫克'], 
                 '鈉':[0,'毫克'], '鋅':[0,'毫克'], '維他命C':[0,'毫克']}
    # try open csv
    try:
        df = pd.read_csv('eatlist.csv')
    except:
        print('目前還沒有任何紀錄可查詢，請先至少記錄一筆資料後再查詢。')
        return
    # 找出日期一樣的
    df2 = df[df['日期']==str(Date)]
    if(len(df2)==0):
        print(Date.strftime('%Y') + '/' + Date.strftime('%m') + '/' + Date.strftime('%d') + "沒有紀錄");
        return
    df2.reset_index(inplace=True, drop=True)
    # 統計當天總熱量及養分
    for i in range(len(df2['食物名稱'])):
        food = str(df2['食物名稱'][i])
        calo = search4nutrient(food)
        for key, value in TotalCalo.items():
            TotalCalo[key][0] += float(calo['數值'][key])*(float(df2[df2['食物名稱']==food]['重量(公克)'])/100.0)
    todf = pd.DataFrame.from_dict(TotalCalo,orient='index',columns=['數值','單位'])
    todf.sort_values(by='單位',inplace=True)
    # 用plt畫表格
    plt.figure(1);
    fig, ax = plt.subplots();
    # 隱藏坐標軸
    ax.xaxis.set_visible(False);
    ax.yaxis.set_visible(False);
    ax.axis('off');
    ax.axis('tight');  # 設置為緊密貼合
    ax.table(cellText=[[round(v[0],2),v[1]] for _,v in TotalCalo.items()], colLabels=todf.columns, rowLabels = [k for k,_ in TotalCalo.items()], loc='center');
    fig.tight_layout();

    # 統計三大營養素圓餅圖
    col = ['醣類','脂質','蛋白質']
    val = [(TotalCalo['糖'][0]+TotalCalo['碳水化合物'][0])*4.0, 
           (TotalCalo['脂肪'][0]+TotalCalo['飽和脂肪'][0]+TotalCalo['反式脂肪'][0])*9.0,
          TotalCalo['蛋白質'][0]*4.0]
    # 畫圓餅圖
    plt.figure(2);
    fig, ax = plt.subplots();
    ax.pie(val, labels = col, autopct='%1.1f%%',shadow=True, startangle=90);
    plt.title('三大營養素攝取比例');
    fig.tight_layout();

---
## 建構interact元件
- 根據RecordEat()建一個互動式表單
- 根據CalculEar()建一個日期選單，讓使用者選擇要查看哪一天

In [6]:
print(' '*16+'每日食物攝取紀錄表')
interact_manual(
    RecordEat,
    Date=widgets.DatePicker(description='日期'),  # widget list DataPicker: 顯示可選擇日期的輸入框
    item=widgets.Text(description='食物名稱',disabled=False,  #widget list Text: 輸入文字
        placeholder='ex:白飯、油麵...'), 
    gram=widgets.Text(description='重量(公克)',disabled=False,
        placeholder='ex:100'), 
    note=widgets.Textarea(description='備註', disabled=False,  #widget list TextArea: 輸入多行文字
        placeholder='ex:九折商品...'),
);


print('\n'*3+' '*16+'查詢當天攝取熱量')
interact_manual(
    CalculEat,
    Date = widgets.DatePicker(description='日期')
);

                每日食物攝取紀錄表


interactive(children=(DatePicker(value=None, description='日期'), Text(value='', description='食物名稱', placeholder…




                查詢當天攝取熱量


interactive(children=(DatePicker(value=None, description='日期'), Button(description='Run Interact', style=Butto…

<html><head>


<!-- Load require.js. Delete this if your page already loads require.js -->
<script src="https://cdnjs.cloudflare.com/ajax/libs/require.js/2.3.4/require.min.js" integrity="sha256-Ae2Vz/4ePdIu6ZyI/5ZGsYnb+m0JlOmKPjt6XZ9JJkA=" crossorigin="anonymous"></script>
<script src="https://unpkg.com/@jupyter-widgets/html-manager@*/dist/embed-amd.js" crossorigin="anonymous"></script>
<script type="application/vnd.jupyter.widget-state+json">
{
    "version_major": 2,
    "version_minor": 0,
    "state": {
        "119061287b544536bdcb05beb95aa0bc": {
            "model_name": "DescriptionStyleModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.4.0",
            "state": {
                "description_width": ""
            }
        },
        "3cc6bc2564e14384bd643adec4fb6de0": {
            "model_name": "LayoutModel",
            "model_module": "@jupyter-widgets/base",
            "model_module_version": "1.1.0",
            "state": {}
        },
        "e69e0131421641a2b9ee6b8eab2acc86": {
            "model_name": "DatePickerModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.4.0",
            "state": {
                "description": "日期",
                "style": "IPY_MODEL_119061287b544536bdcb05beb95aa0bc",
                "disabled": false,
                "value": {
                    "year": 2019,
                    "month": 3,
                    "date": 15
                },
                "layout": "IPY_MODEL_3cc6bc2564e14384bd643adec4fb6de0"
            }
        },
        "1833eaa3ba36472ab7887fca139f2a88": {
            "model_name": "DescriptionStyleModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.4.0",
            "state": {
                "description_width": ""
            }
        },
        "ea91f3f4003140bd9431f143775f87c8": {
            "model_name": "LayoutModel",
            "model_module": "@jupyter-widgets/base",
            "model_module_version": "1.1.0",
            "state": {}
        },
        "e6da1ade59b74cd1bbe78e4ff4bf99f5": {
            "model_name": "TextModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.4.0",
            "state": {
                "description": "食物名稱",
                "style": "IPY_MODEL_1833eaa3ba36472ab7887fca139f2a88",
                "value": "飯糰",
                "layout": "IPY_MODEL_ea91f3f4003140bd9431f143775f87c8",
                "placeholder": "ex:白飯、油麵..."
            }
        },
        "c286198e2c0a4a2f91f4f6b91f6cb599": {
            "model_name": "DescriptionStyleModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.4.0",
            "state": {
                "description_width": ""
            }
        },
        "988274bbb6f54a7d98827caa8810f5d0": {
            "model_name": "LayoutModel",
            "model_module": "@jupyter-widgets/base",
            "model_module_version": "1.1.0",
            "state": {}
        },
        "9264d268a64c4f248946870570c95b0e": {
            "model_name": "TextModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.4.0",
            "state": {
                "description": "重量(公克)",
                "style": "IPY_MODEL_c286198e2c0a4a2f91f4f6b91f6cb599",
                "value": "250",
                "layout": "IPY_MODEL_988274bbb6f54a7d98827caa8810f5d0",
                "placeholder": "ex:100"
            }
        },
        "d0f3dbfbc40d465a9e87b42694f6406d": {
            "model_name": "DescriptionStyleModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.4.0",
            "state": {
                "description_width": ""
            }
        },
        "c59aec404335431c9c6f048c1df7f688": {
            "model_name": "LayoutModel",
            "model_module": "@jupyter-widgets/base",
            "model_module_version": "1.1.0",
            "state": {}
        },
        "2c629bbb562746a7bcff895ec4bf9072": {
            "model_name": "TextareaModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.4.0",
            "state": {
                "description": "備註",
                "layout": "IPY_MODEL_c59aec404335431c9c6f048c1df7f688",
                "style": "IPY_MODEL_d0f3dbfbc40d465a9e87b42694f6406d",
                "placeholder": "ex:九折商品..."
            }
        },
        "623ca3f9d46b45dd992b8d45ab633275": {
            "model_name": "LayoutModel",
            "model_module": "@jupyter-widgets/base",
            "model_module_version": "1.1.0",
            "state": {}
        },
        "7971196dea494f95ac9c502564cdf669": {
            "model_name": "VBoxModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.4.0",
            "state": {
                "layout": "IPY_MODEL_623ca3f9d46b45dd992b8d45ab633275",
                "_dom_classes": [
                    "widget-interact"
                ],
                "children": [
                    "IPY_MODEL_e69e0131421641a2b9ee6b8eab2acc86",
                    "IPY_MODEL_e6da1ade59b74cd1bbe78e4ff4bf99f5",
                    "IPY_MODEL_9264d268a64c4f248946870570c95b0e",
                    "IPY_MODEL_2c629bbb562746a7bcff895ec4bf9072",
                    "IPY_MODEL_568bd807d0374d87ab7503d0443528e1",
                    "IPY_MODEL_0fa2fe42e77b46a0856a835d15680b9e"
                ]
            }
        },
        "59b7f43edbbd411599e5773746d9e9b2": {
            "model_name": "ButtonStyleModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.4.0",
            "state": {}
        },
        "5bd394bacfb0415a9bc18286337c6e71": {
            "model_name": "LayoutModel",
            "model_module": "@jupyter-widgets/base",
            "model_module_version": "1.1.0",
            "state": {}
        },
        "568bd807d0374d87ab7503d0443528e1": {
            "model_name": "ButtonModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.4.0",
            "state": {
                "description": "Run Interact",
                "style": "IPY_MODEL_59b7f43edbbd411599e5773746d9e9b2",
                "layout": "IPY_MODEL_5bd394bacfb0415a9bc18286337c6e71"
            }
        },
        "88f69369da9a4201b9e1bb2bdd17ae39": {
            "model_name": "LayoutModel",
            "model_module": "@jupyter-widgets/base",
            "model_module_version": "1.1.0",
            "state": {}
        },
        "0fa2fe42e77b46a0856a835d15680b9e": {
            "model_name": "OutputModel",
            "model_module": "@jupyter-widgets/output",
            "model_module_version": "1.0.0",
            "state": {
                "outputs": [
                    {
                        "output_type": "stream",
                        "text": "      紀錄成功！\n",
                        "name": "stdout"
                    }
                ],
                "layout": "IPY_MODEL_88f69369da9a4201b9e1bb2bdd17ae39"
            }
        },
        "c5d84867ef58453bb4cb4b0e2a47d5e4": {
            "model_name": "DescriptionStyleModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.4.0",
            "state": {
                "description_width": ""
            }
        },
        "a8bc52baa4664411b122d6f22f0311de": {
            "model_name": "LayoutModel",
            "model_module": "@jupyter-widgets/base",
            "model_module_version": "1.1.0",
            "state": {}
        },
        "befd4b5e6fd34f46b94f2460b21e1fe9": {
            "model_name": "DatePickerModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.4.0",
            "state": {
                "description": "日期",
                "style": "IPY_MODEL_c5d84867ef58453bb4cb4b0e2a47d5e4",
                "disabled": false,
                "value": {
                    "year": 2019,
                    "month": 3,
                    "date": 14
                },
                "layout": "IPY_MODEL_a8bc52baa4664411b122d6f22f0311de"
            }
        },
        "9130327998f34d7c9f963a28ebe37c77": {
            "model_name": "LayoutModel",
            "model_module": "@jupyter-widgets/base",
            "model_module_version": "1.1.0",
            "state": {}
        },
        "4b352cf198fc46cb92952aa63b68e8e8": {
            "model_name": "VBoxModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.4.0",
            "state": {
                "layout": "IPY_MODEL_9130327998f34d7c9f963a28ebe37c77",
                "_dom_classes": [
                    "widget-interact"
                ],
                "children": [
                    "IPY_MODEL_befd4b5e6fd34f46b94f2460b21e1fe9",
                    "IPY_MODEL_3daf3b8cdebe4cedb4d0588c73cbaa56",
                    "IPY_MODEL_77ba7615d45c4b5eac8201d58ccadf8a"
                ]
            }
        },
        "5f2f71e14831489e9bbe8138280aa57a": {
            "model_name": "ButtonStyleModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.4.0",
            "state": {}
        },
        "4b00c9e24a4344ac8c5a30066fb7f475": {
            "model_name": "LayoutModel",
            "model_module": "@jupyter-widgets/base",
            "model_module_version": "1.1.0",
            "state": {}
        },
        "3daf3b8cdebe4cedb4d0588c73cbaa56": {
            "model_name": "ButtonModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.4.0",
            "state": {
                "description": "Run Interact",
                "style": "IPY_MODEL_5f2f71e14831489e9bbe8138280aa57a",
                "layout": "IPY_MODEL_4b00c9e24a4344ac8c5a30066fb7f475"
            }
        },
        "12810ce0ba8e479183f1a775fb1e9abb": {
            "model_name": "LayoutModel",
            "model_module": "@jupyter-widgets/base",
            "model_module_version": "1.1.0",
            "state": {}
        },
        "77ba7615d45c4b5eac8201d58ccadf8a": {
            "model_name": "OutputModel",
            "model_module": "@jupyter-widgets/output",
            "model_module_version": "1.0.0",
            "state": {
                "outputs": [
                    {
                        "output_type": "display_data",
                        "data": {
                            "text/plain": "<Figure size 432x288 with 0 Axes>"
                        },
                        "metadata": {}
                    },
                    {
                        "output_type": "display_data",
                        "data": {
                            "image/png": "...\n",
                            "text/plain": "<Figure size 432x288 with 1 Axes>"
                        },
                        "metadata": {
                            "needs_background": "light"
                        }
                    },
                    {
                        "output_type": "display_data",
                        "data": {
                            "image/png": "iVBORw0KGgoAAAANSUhEUgAAAQ0AAAEYCAYAAABGCaMgAAAABHNCSVQICAgIfAhkiAAAAAlwSFlzAAALEgAACxIB0t1+/AAAADl0RVh0U29mdHdhcmUAbWF0cGxvdGxpYiB2ZXJzaW9uIDMuMC4zLCBodHRwOi8vbWF0cGxvdGxpYi5vcmcvnQurowAAIABJREFUeJzt3Xl8lNW9x/HPb7bMTFYgCWEPYACXiAqauiBad1utWmutXai2tfum7ZVe20pve6u9XbW1rq2lrVtrNxTrghYVF0DWoIJhh0DIvk4y67l/nCcSY0IyMJnnmeS8X695hcz6y5D55pzznOccUUphGIYxWC67CzAMI7OY0DAMIykmNAzDSIoJDcMwkmJCwzCMpJjQyDAi4u71vUtEgnbVY4w8JjQcQkTKRGTiIW6fJCK5QLWInCQil4rIMcAJwFYrPK4VEen1uNEikt/j+wkikjWIen4sIl88xO0fEZFbrdf19HOfo6yv0vs+IuLpHYCDqCkiIv5D3H75ADXfLCJjk3lN471MaKSQiFwiIs0isrefS4uIvL+fh38Z+Mohnv736IBwA18CaoHfAucC9cAXgZOVNfFGRHzWB3Uu8GiP53kAONP6sPt61H6xiKwTkRUisgL4BHBD9/ciskZELu/xPF1AGDgKeFJEakXkiR6XA8Bt1n2PAtaLyH7ruTYA+4D7RWS9iLxuXR7tUc99InJtr/cgAkQP8R59CugUkWVWPS+KyJ09br/Qev+MI9DnXwjj8CilHgcKDvPhYaD6ELd/Bh0qrwO7rK8fAf4KPAV8Dvhsj/t/F6hAf8hiIrIJ/UGvAb4OfBNYDdxi1f6kiNQBfkAB16GD6QlAgE6l1OsAIjIGGAeMtq4/X0TWAst6vL5XKXWl9dxVIvIV4Gql1BdE5H3AF6zLKqXUXBGZBfymx+P7CoiYUipu1TANmKOU+qv1/WTgbOBjwF+A9UqpM63bXrPemziQOMR7bAyCCQ2bWF2G9u4PAfr/orPH7VlAEGhRSiWAHwFe4HLge8AxwP3Aw0qpX4rIX4C7ROQspVQH8H/AIqAJ/WGZDvwb6EC3MMO8+0MKcDRQiP5gjbdef651/1p0UAGcAlwKFAFXALcDo4DLejxXnPf6kIgcB+QC65RSYREJWLeVAjt73LfPqcoikg18Dd0y+3WPm25Bv59dInISOig/AVQBbdbPbaSACY0UExF3jyDoeb0AYgUAwCbAJSLdH45RQFREFlnfu4AsoNwa6zgGaEe3KgBOt+7zIRH5kHVdB/qv7RPAPcAYIAacAWwH3gfMA15E/yU/C7jE+pDdh/5wgf69mIoOnJIeP8INwAeVUv+2xiPmAq+IyB3AW8BadFjEgAUi8j3gF1aIAfyrV0sD62d2AccBG3u9bXeIyE+AHA523TYCDwPHK6UarcLORnfTuqz7XI5uRb0PONaqyZwvkSImNFLvLusvXKTX9T7gB8BPAJRSk3reKCLPAbuVUr378YjI0ei/8jeiQ6Evy9EfrLHW0ZQ70GFxGrorsgb9wfo+cAO66b9TRE5TSr0CzLHC6YPW89wJrADOQwfIgj7CcCpwJroVkwBOAp4DLkEH0DnA34A3+6kZ4G1gBjrAvt/rtq8ppf7c4334NToserca9gPXA7eLyAR0V+596PCIAice4vWNJJmB0BRTSl2vlAoqpQp6XYJKqZ/09Rjrw1oKzLX65r2tB36F7jrciR436Xn5OTAW3U14ED1g+iv0WMRFwAJ0iFwK3ArMB55Bf9j293idXGCK9RwfA36KbhHdDny1R71z0V2E6cBSdMvibuv17kD/xf8e0Axs7vH8V4jI6+jB2G7LgGuBGUqptX29Pz31ERgopTYrpZ62vj0D3cr6h/VzrxroOY3kmJaGM/wCHQZ7gHvRo/zvUEo1AA1WuNTw7gFP0B/OmFKqe0zkFeBUEckB/gvdXekpHz1QuL77Cqt1cim69XABEEIHyo3oYCoVkQ6l1H1AA/AvoEgptVlEurtcCaVUtMdRX9WjOwbw9x7dkxtE5ETgz8Be4L/7e3OsrtCgBpiVUo9aj/kbOpDWoAd3zVGTFDGhkSJ9jFn0d78sIK6UilnfXw9MBj5ufeA+IiK3KqW+08fDrwee7eepPyciv1JK1fe6Pgfd5QD9/x1Ddx3miUiXUmozgFIqZB2i7QA+bdU0H7gamIkOkKnWfXeIyC70QCjocY9vcfD3abz1fcT6Gf3Wc5woIvcBk6znP8p6zjrgwyJyn1Kqe1yi+/3qbh39tPcPLCJXAU8opUJ9vB+fAe5XSinrPd7bz/tmJMmERupcDPxDRDo59KBbNnCuNVfhf4HzgbOVUt2HF68DlotIKfB1pVRtj8d+Hj1O0JcHgd+h52z0tBt9aHYauosRBgLo7lATvPOX/AL0IVoPujsUQY9RKOA7wER0l6ebG1AiMg99lGUFcKmIfBU9frPa+v549KHkC9DdqOfQAfJ56363oAd1Pw6sEpFPW92UGcA16LGczyqlXhWRn4tIiVKqRkSmo1sp49GtIgCPFd7fsF7vZOv6fehwKuK9Y01GspRS5pLmC3owswk96Wp0H7cXAI+huyKje1zfiR7w7OvSih4X6Pk889Fdl8eAc63rxqAHQl8HPmld50b/NT/V+v5MoBH4co/nOhH4So/n2IYOyiB6TgbA09Ztedb3T/bz85+JPgLyKWBKj+svAcZZ/74OuLDX436MHlTdbH29qdftB9Bh+Dwwqcf1eeiu35/t/r8fDhex3lQjjaxDjKcrpV4a4H7HKaU2HcHrZAEe1cfgodUVyVPWYcs+bveqg62fQd0uIgF1cFzFGKZMaBiGkRRzyNUwjKSY0DAMIykmNAzDSIoJDcMwkmJCwzCMpJjQMAwjKSY0DMNIigkNwzCSYkLDMIykmNAwDCMpJjQMw0iKCQ3DMJJiQsMwjKSY0DAMIykmNAzDSIoJDcMwkmJCwzCMpJjQMAwjKSY0DMNIigkNwzCSYkLDMIykmNAwDCMpJjQMw0iKCQ3DMJJiQsMwjKSY0DAMIylm1/gRrnTh0lygBCgG/OjfCQ/gCU77edydVecC4ujd1lvRG1c3A02VCyr73evVqaxd5T2H2qe21/3dwByl1Kp+bs8GCoFjgHZgBfp9HA9MVEq9kJLCHcSExjBWunCpHygHjgcmA2PRATG2xyXY3+Nd3pbNwKz+bi9fXB5C7y6/B9hhXXb2/HflgspECn6UVBoH3C8iEev7GeifoR79YT8RKO2xkfV5wNVAn6EB/APYh96xfi3wIPo93gHUi8grgw2oTGE2gB4mShcuHQ/M7nWZAbgP9zkDk26u9+TEC4+grA5gI/rDtM76+kblgsrIIR+VBiLyHeBJ4BIgALwFjAE2K6WeFpFPAdcBpUALuoXV7UWl1Pet53kOiAI+dIusDRht/VuA/1FKLU/Dj5Q2pqWRoUoXLp0GXGBdTkc3kVNLqSMd88oGTrUu3SLli8vXAM9bl5crF1SGj/B1kiIiZcD16O5EENiPbiFVc7BFMQH4jVLqsV6PLQV+Yv3bA7iUUheKyFPAlcBSpdR8EfkjcKNSqm7If6A0M6GRIUoXLs0GzgYuVEpdICJHDfmLigxF18LHwSC5GegqX1z+CvAc8GTlgsr1Q/Ca7xCRs4CfAvPQrbFzrVouRYdcsYj8HN166E938/x04E4RuRbYBXwTWCcil6FbGUcBwy40TPfEwUoXLp0AfAy4WCl1uoj40vn6KeieHI5twGPAY5ULKl9P9ZOLyAygAB1eAB8FNgMbrO/fVkrVisi3gM8ADb2eIgvYppS6usdzHkB3b3oqBO5QSt2b4h/Bdqal4TClC5dmAZcppa4DzhURF4Ae9B8RpgM3ATeVLy7fCfwN+FPlgsoNh3zUICml3haRjwBzgKfQ3RMv+rNwIbAeeMS6+/f66Z7c1utp9/d4TLfZgNMGgVPChIZDlC5cOlcpdS2oj4u48kdQSBxKKXAjcGP54vJVwD3AI5ULKkNH+LwCXAa8D5iCDpBmIJ/+j5Icynj0EZaeCtGDwMOO6Z7YqHTh0lHAtSqR+Ky4XEfbXU9vNnVPBtIK/Bm4p3JB5WF9KEWkCP0HsxX4b2ALuoVxO/A5pdRWEfk2+uhJX92THUqpq6zncgFnAF8C/lcpVSkiV6DHSu5TSq07nBqdzISGDUoXLi1Sifi3EPmSiCvH7nr649DQ6Ok54NbKBZXPDfYBIjILeAM9N6PZurRYl1bgZHQL5Bvow699dU9+oZS6wvr+h+ijMA8ppfb0uN9c4CvAj5RSWw/z53MkExppVLpw6fhELPJdcXmuE5cry+56BpIBodFtJXArsKRyQeUhf6GtGaFupVSsn9vHKKUaRCQIxJRSts8pcRoTGmlQunDplEQscou4PZ8QcXntrmewMig0um0Cfowe9zC/2EPEhMYQKl24dFIiGv6JeLxXibgOe2amXTIwNLqtB26qXFD5jN2FDEcmNIZA6cKl3nhX2y0uX/Bb4nI7vhvSnwwOjW7PADdULqh8w+5ChhMTGik26St/vkR8/ntcvsA4u2s5UsMgNECfA3Iv8N3KBZWNdhczHJjQSJHJ33h0klKJxe5A3tl215IqwyQ0utUCX61cUPkXuwvJdCY0jlDpwqXeeHvTD1yB3BvF7UnrNO+hNsxCo9sS4EuVCyqr7S4kU5nQOAITrr+3wp096jFXVnCi3bUMhWEaGqDnY9yEniBmPgBJMsv9HYZgWYVr/Gfu/KmnoOTl4RoYw1wecBewrHxxecaPPaWbCY0kFV/x3Umjz/38Wl9R6bfE5c64w6jGu7wf2FC+uPxCuwvJJCY0klDy8Z9c7Z8y+w1P/tjZdtdipEwR8GT54vL/K19cnjET7+xkQmMQRp11rXf8dXf+KWvCMQ+5soK5dtdjpJwA3wZeKl9cXmpzLY5nQmMARR/6r7LsY89+y1dc+glxucz56sNbBbC6fHH5PLsLcTITGv0IllVI4QdvvDQwdc5qT+6Y6XbXY6RNIXqA9NN2F+JUJjT6ECyrcGcfPf+7wVmn/8Xlz8m3ux4j7XzAA9Y4h/mM9GLekF6CZRX+nNkX3BecdcYilycrY88bMVLi28DfyxeXZ9tdiJOY0OghWFZRkDvn0r8Hpp9yrbjc5r0xAD4EPFu+uLzA7kKcwnwwLMGyipK8iiufDpSecJFZn9Po5VTg+fLF5cNxdmzSTGgAwZmnlRac8fFl/onHnGJ3LYZjnQi8YGaQmtAgWFZxdH7FlU/4xk4/1u5aDMc7Bj2XY4rdhdhpRIdGsKzi2Ly5lz2cNX6mCQxjsKYDL5YvLp9kdyF2GbGhESyrKMs54eIH/FOON1PCjWRNBp4ZqWMcIzI0gmUVpdnHnfO74PS5J9tdi5GxZqHPWXHsFhRDZcSFRrCsYkJw1rz7gjNOM1OFjSN1MvDP8sXlI2o+z4gKjWBZRXHgqIq7so+Z/35zWNVIkXOAh0bSzNER84MGyypG+8bN+FlO+bkXi7hGzM9tpMUVvHdT6GFrRHx4gmUVua5g/nfz5lxymVk4xxgi3y5fXH6N3UWkw7APjWBZhRtxXZ9/6kevcmVlm7UwjKF0f/ni8jl2FzHUhn1oAJfkzrlkgbegZILdhRjDXgD4R/ni8mK7CxlKwzo0gmUVswPT5n7NP/n4crtrMUaMScDfyheXe+wuZKgM29AIllWUeEZPXJhz/HmnmyMlRpqdAfzA7iKGyrAMjWBZRUB8wRvy33fl+eL2DqsNjIyMsXC4Lhs47EIjWFbhAhbkzf3Qpe5A3mi76zFGLBfw5/LF5cNu5bdhFxrAOVmTjrvcV3LUTLsLMUa8ycDddheRasMqNIJlFePFk/Xx3NkXnmzGMYa3SEPE7hIG6+ryxeWftLuIVBo2oREsq3AD1+bOuWS2Kys4yu56jINiLTG2/3j7u67b9ctddO7q7PcxkYYI22/dzo6f7KD6gWqUUtT8tYadP9uJUoqOzR1DXXYq3VG+uLzE7iJSZdiEBjDfV3LUqVkTjjanujtIvCPO3vv2kggn3rmu+ZVmfMU+AlMC/T6u6T9NjF8wnqk3TSXaGCW8N0y8LY5/ip+uXV14R2fUZmgFwO12F5EqwyI0gmUVRbg81+Se9MFTxPRLnMUFk740CZdf/6rF2mPUPFKDO9tN+1vt/T5s7JVj8Y/3AxBvj+POcaOUgjh0bOkge1ZmLRAuSp3/z5+NP8/uOlIh40PDOlryqdwTL57tDuQV2V2P8W7ugBt38ODpPg1PN5B3ch6jzhpF88vNtK5rPeTjW1a2kDUhC+8oL/6JfiINEcQl7Lh1B137uoa6/JSY2hFe+/ye6uhl7R13syjfb3c9RyrjQwN4n2f0hHn+ycefaHchxsC6dncx5pwxeAu85J+Sf8ixiUhthPqn6hl3jV7Lt/CCQgpOK0B8Qt6cPNo39N9ScQJXKNby7bcPtC+pPXBSYTxRBEwDFtpd15HK6NAIllWMAj6ZO/vCWeJymbNXM4Cv2EekTh/56NzRiXdM32MT8Y44e+7ew4TrJryrpRIPxXH73YhHdHfFgVRCxU/Z3VL78r7qvE95w71X9rqJRflTbSksRTI6NICP+ErKxnpHTzBzMjJE4cWFNDzXwPYfbafj7Q5GzRtF+5vtNCxreNf96pbWEW2Isu/P+9h+63Y6NncQrgkTmBwgMC1A47JGsmc6b1wjtylce/+2fbHfxVuKc9x9jq/5gf9Jd12pJE5N64EEyyomA/8z+rwvzvPkFU2zu57hKDDp5npPTnxELp6btEgidM2exq7vuEKDmYWcAI5nUcsbQ13WUMjIM/GCZRUCfNg/+fjRJjAMOymlmFYTOnB3R0PheDfBQT7MBfwQveJXxsnU7slRwOzsY846ye5CjJHL0x5r+uHWAx1LuhrGjneT7Jja5SzKnzskhQ2xjAsNq5VxVeCoiiJ3doFZWMdIOxVX0bN2NtW9eqB61OWeyJEMrPxvyopKo4wLDeBYRMqCM083e5YYaTe6vuvAI9ur1a9VW5HfdcTzCM9nUf6ZqagrnTIqNKzzS64OzjxjrNufYyZyGWkj4Xj79VvrWl5oqx17nCeRyjVavp/C50qLjAoN4ARgYqD0hOPsLsQYGVRCJY7Z21azfE918KvuzqFYG+McFuVn1F7CGRMa1nTxq3wlZW539qiJdtdjDH9ZrZH6X27dH3402lQy2j2kn5WvDeFzp1zGhAYwAygOzjgto1LZyDwqlgh/YHtj/av1+wvP88b6PxU3dT7BovyMWc4hk0LjHJc/N+EdM9GsLG4MmZLaUM0/d1S7b5P2Qm/6TpgOAp9N14sdqYwIjWBZxRhgTvYxZ00SlzsjJ6QZzubqjLXcWHWg7dmO+pKjPMqO37Evsyg/I86fypQP4GmAyho/c9jvXmWkl0qo+Ml7Wxt+HWkuyvHYuhbLFOBS4B821jAojm9pBMsqfMAFgekn57iygmZ1cSNlspvDdfds3Rd7oP+Ty9LtOrsLGAzHhwZQDuT4S08yy/gZqRFNdH50W0PTa00Hik73xrPsLqeH8zNhQNTRoWFNGb/YlZXd6ckrOsrueozMppSidH9Hzb93Vfu+6+pw4ofTRwacxObo0AAmANMC008uMovsGEfC0xFr+sHWAx2PdzWUTHQrJ/8uXW13AQNxemiUA8o3dvosuwsxMpOKq+iZO5vrXq2pHvXhIzu5LF3OZlH+WLuLOBTHhobVNTlDPL4WT/7YMrvrMTJPQUPXgT9vr1Z3qtZUnFyWLm7gI3YXcSiODQ2gCBjvn3pSsbg9ZhNnY9AkHG//zNa65pdaa8eekNqTy9Llo3YXcChODo1jAZU1bqbpmhiDopRSs6rbap7fUx38hruzwO56jsBpTj6K4uTQOB2RVs+ocWbRYGNAWW3Rhp9tren6a6SppHBoTy5LBxdwtt1F9MeRb26wrKIAmOaffHyey+PLhMErwyYqlghftKOx/tW6fWMu9ETTcXJZupxrdwH9ceo08lkAvnEzpttdiOFcY2s7D9zZ0jB6pidRiCMmdKbUOXYX0B9HtjTQ55p0ePKKJtldiOE8rs546zeqaluXddSNnelJZNRO0EmYwaJ8R/7+Oy40gmUVXuAYRJrcwYLxdtdjOIdKqPiJu1tqX9q3N/cznq48u+tJA0d2URwXGsA4QHxjpxeK2+Ok8wIMGwWbI3V3b9sf/WO8pTjPNfz6Iv1wZBfFiWMaEwHxFU8zS/oZEE10Xrm7qfMWV0eRI39bh9b77C6gL05saRwNhD0F40xojGBKKSbXdNQ8uavad4urY6QuiTCNRfm5dhfRmxOz+2ig1Z07xoTGCOXpiDV9Z1+99ypPpCTpfcuGFwGOB162u5CeHNXSCJZV5AGjXf7chCsr22w8PMKouIqdvrO59uWa6oKrPJEcu+txiBPsLqC3AUNDRPwiEhARj3Xx9rj+C4N5ERFxichgAmoielm/8SIjZrDLAPIbu2r/uH1f4m7VWhwcOQOdg+G4xacG0z35HPB5IAdoBR4UkYus244TkWuA/wBlwFagBpgOTLW+BoH9wHeBFwd4rVIAT0GJo08NNlIoHG+/dk9j7AZ3Z7EjO8v2c1xLY8D/JqXUr0VkBzAXWKGUWiYic5RSV4nIV4E70TM4L0H3v/zA28AfgOuBTmAtsG4Q9RwLtLuDBSN14GvEUEqpmfvaa+/qbCoqzvxzRYbScSzKd7OoJW53Id0GDA0R+TS6tdEJnCUiJUBERG4ArgIuAP4PKAHmA38FsoAPA6uBfejweBVoG+DlJgOtrkCuCY1hzNcWbfjh/vrAxd7o2BE+0DkYAfRK5dvtLqTbYBuElcA2IB+IoY8fTwQmocPiPnQo3AvsAq4EVgFfRofNdOCQSRksqwgA2UCDKyvbhMYwpGKJ8AV7mttuU22FXq8ZtkjCRBwUGoNtFp6E7nKcaH2/HPg5sBEdDucD1wINwGjgV4ACfgosZOCxDKzHJRAR8fmHYqNdw0bFdZ0HHtuxz/Vz0rpz2XDhqHNQBtvS+AP6WPEU63tBdzs6gV8AXcDt6GmvK4FRQAS9MLAHGMx5AqMB8eSNzRFxZXwfN9ZahyevyO4ybCdd8dav7GlQ13u6xpqBzsPmqDlLg/lwCnoE97PAcYAX3UW5D/1B/w36yMq3gYR1/Q/RYXEN8AXr8QMtuzYKcLnzCm05ESne0cS+Bw5u3t3wzG8JbV054GNqHvyvd75veuEPHHj0eyil6Nq9cchqzQQqoRKz97QeeLG6Ovd6T5dpOR6ZjGtpCDocQtbXMmAHEFdKfUlEvoXuinwT+DfwSWAe8Bh6dudC4Db0WMehFAFRd/YoW37Bmv7ze1QsAkDXnk3EO5oIHlXR7/3jXe3UL/0lKhp+57pEqBVfyXQiB7aN6FZGoCVS99MD9bnzvbGx5rhISmRcS8MN/EYp9Q10qyEbeAm4RESWA18EzgMKgYVKqS8BfwRuBv5LKbUDPX9joOXLxgJhdyAv7S2Nzl0bEG8W7uwCVDxGw1O/wZNXTKjqtX4fI+Ki6EM3Ib6Di0UppSCRILznDbImjcDN7aOJziu2NTS+2rC/aL435re7nGHEUaExmHka9/X4d72IfEsppThEk0kpVQN8rMf3Nw+iliIgLD5/Wn/ZVDxKyyuPUHz5zdT+/Ue0b3oeb+Ek8iuupHXt48Ra68ibc8l7HufKCr7nOl/RFMLVm3HnFnLgoZsYc8FX8RY6qmU5ZCbVhGrubmsomuxRAd04NVLIUevKJN14tAJjKIwBIuLypHW4rOW1x8g98QO4/PpUh2jtNnJnX4g7ZxQ5x5yd1NhE3smXkX3c2Yg3i+CM0whtWz1UZTuGOxRr/u+qmvYnO+tLJnscvXNZJnvvXygbOWk8Owg043Kntaaunevp2rWBtrVPEKndQXjvm3jHTAYgXFOFJ684qedLdHXg8gVIxGOoROdQlOwIKq5ip+1tafplrKUw22OOoQ4xRy2Y7IhhqmBZhQs9dpIQlzutaz6WfPwnlFxzGyXX3IaveCqTvv4IXbs3UvPgTbStW0reKZcTqd9N04t/GvC5oo3V+Iqn4hs3g9Y1j+MfpuMaeY3h2j9s25e4N9FalG1OLksHH4vyHfFZBZCh620MnrUu6D3A7oL5n77MVzjZcWf2jUSBSTfXe3LiB5coiCQ6PrW7MfJtd8ixG/kMYzksaumwuwhwTvfkndaFpLl7YgxMKaVm7O84cFdnY/FYN2YfGnsEABMaPXjQcz0gzQOhxqF526ONP9hfn3WJJ1rijM7siOWYwVCnfEDfqUNcLqfUNOJdsac1fouvvtDnMcMWDuCYwVCn/O3o0dIw3RMn8MdbE4t89cU+lwkMh3DMehpOCY2DR0xUwjFvzkh2ZvilFq85MuIk4YHvkh5OCY13WhcqFnXMmzOSXeBa02J3Dca7ROwuoJtTQiPW/Q8Vj3TZWYihnZK1wzEDbwagl59wBKeExjutC9PSsF9OvCUywd9ltpBwlna7C+jmpNAQABWLmNCw2dmxFXWuwW05YaRHyEkLCzvlF6MLqxYVDTumGTZSncuq2MD3MtJooAW508opoRFDH1ISFe0yLQ2bnezbZWZ9OkuD3QX05IjQCFWtVOj1Rj2JWNiEho3y4s3hEn94jN11GO+yx+4CenJEaFhCgEeFQ6Z7YqP3R1+sd5ktMZ3GhEY/QoA71lZv5gfY6FxZHbW7BuM9TGj0ox3wRBv2NCmVSNhdzEg1N2t3rt01GO9hQqMfNYCfRDyhIp1NdhczEo2KNXSNzYqY8QznMaHRjz3oPWBJdLU7arR4pHh/7MV6M5zhSCY0+lGPdSZfPNRqQsMG58rrjplAZLwjAey2u4ienBQa7wRFvKPJhIYN5mTtybG7BuM9qljU4qgVqp0UGo3oeiTWWmdCI83GxOpCRVlRM57hPOvtLqA3x4RGqGplFN3ayIo1VpvQSLPzYssbzXiGI62zu4DeHBMalmogGGupaVPxmJkZmkbvl7VmPMOZTGgMYDfWWojxjqa9NtcyopyYVZ32PXSNQTHdkwFUozdNItZcM9Au80aKFMdqQkX+mNnLxHn2sail1u4ienNaaOzFWmA4UrvdhEaanBd7wYwhOdNauwvoi9NCYz+exCl7AAAUcklEQVR6LURPeO+b1SoRN+s6pMH7XWvNtH1nWm53AX1xVGiEqlYmgLeAfBWPxuOhlmq7axoJTszal293DUafltldQF8cFRqWDaC3/jPjGkOvJLavY3RWvMDuOox3U0rVAhvtrqMvTgyNnVjjGtG6nSY0htj5seWNdtdgvJeIPMeiFvt3Z++DE0OjGn0Oirtr7xt7zGnyQ+v9rnWO/MU0eNbuAvrjuNAIVa2MAW8DeSrSGY13NDvqDL/hZrZ/vxnPcCZHjmeAA0PDsh7IBYgc2PaGzbUMWxOju9tG+RImNBxGKbWFRS2O/WPp1NDY2v2Pzm2r3lRKmSb0EDg//kKz3TUY7yUij9ldw6E4NTR2Ay1AIN7W0BFvb9hhd0HD0dmu9RkXxo2dime3xagPpW6o60B7gmjcUW/FQ3YXcCiODA1rvsYLwBiASM1W00UZAsf7D2TUodamTsUHHwqxqjrO2YtD/PCFMGf9oYOz/tDBCXe38/nH+152oqlTcfGDIebee/A+v1kV4eT72umIKJ7eFsPrdsYZvgmlKlnU8qbddRyKI0PDsg6rvtDWVW+ZoyipVRrd3prvS2TUSWobD8T5xQVZ3HxmFhdM93DKBDfLP53N8k9nM2+ym8/N8fX5uD9tjPDxci+vX59DW0Tx+r4462vifPZEH6v3xcn2OiMwAFwijm5lgLNDYzd6YZ5gItTcGW+t3253QcPJ+fEXM248Y36ph/dN9PDirhirquOcOskNQHVrggMdirnj3X0+bkzAxabaOM1dij2tikl5glIQTcAz22JcVOZJ548xkEfsLmAgjg0Na9e1d7oo4Zqtm+ytaHg5y7XBOX9ek6CU4tFNUUYFBK/123vn6ghfnNt3KwPgjMludrUkuGNlhKMLXYwOCOdP9/DE21Em5rm49OEQ/9lh/2lOCaVeY1HLTrvrGIhjQ8OyDms3+c5tKzebE9hSp9xfm1HjGd1EhDs/EOD4YhdLtsRIKMV/dsY5q7TvVgbAD14Ic/cHA3x/fhazCl08sD7KR4/zsuisLAr8wgfKPPztLfv3iHKJ/NnuGgbD6aFRDdQB2YnOtnC0Ya8j5+JnmunRqpZcn8q4TZF+siLMHzdEAGjuUhT4hZd2xamY4OZQSxU2dSkqD8SJJxQrq+N037OqIcH0UUKWR0jYfPAkoVQH8Cd7qxgcR4eG1UVZjtVFCW1Z8ZqtBQ0TmTieAXD9HB9/2hjlzAc6iCs4f7qbp7fFOHPKwVbGm3Vxvvv8u7cD/s4ZWVz/RBf5t7XR2Kn4WLmX1rCiJMfFMUVu7l0T4dxp9o5rKMXvWdTSamsRgyROnzcVLKsYDfwU2AfER5/3xU948oqm21xWRns0+vU9Fbl1k+yuw9CUUgkRmcmilq0D39t+jm5pAISqVjYCrwDFAJ3b17xqb0WZ7zh//Wi7azAOiiV4NlMCAzIgNCzLsLZs7Ny2alu8q73O5noy1szo5uZsr8q2uw7jIK9bfmZ3DcnIlNDYDWyh+/Drnk2mtXGYzou/1GJ3DcZB0bjawqIWx57R2peMCA1rQPQJrDNfO956oTIRi3TYW1Vmmu+uzIj/85HC7eI2u2tIVib9Ar2JPvyao6LhWKSmarXdBWWiY/0NZutFhwjH1E6XSEYcZu0pY0IjVLUyDjwOjAZo3/jMayoWCdlbVWY5JrKpMehVQbvrMN7xHRa1ZNzOdhkTGpbVQBjISnS2hbt2b3zB7oIyyQWJFW1212BooajakuWRR+2u43BkVGiEqlZ2Av8ESgDaNjz9eiLcYTb6GaR57k39z7U20sot3OjUhYMHklGhYVkONAM5JOKJUNVrjl2A1VFUgqMDjWY8wwE6Impd1o9al9pdx+HKuNAIVa0Mo1c2KgIIbXl5S6y90Wx1MIDyWGVjwKM31zbs5XPzdbtrOBIZFxqWNcAOrHkbHZuee9rp0+HtdkHcjGc4QXOXWur9YetLdtdxJDIyNKzlAB8C8gAJV7+1P9ZYXWlzWY42z/Omo1aaGYnCMdXp93Cd3XUcqYwMDUsV+mjKWIC29f9ephJx+xdFcCBRcWb6m8x4hs1qO9QP/T9qrbW7jiOVsaFhzRJ9DPAB7ljz/tauXRuft7ksR5od3Vjv9+C3u46RrKlTbZmU78q42Z99ydjQAAhVrawBngHGA7Ste2JlvKPJsZvM2OWCxAoz5d5G8YRKtEfUxzP1EGtvGR0alseBNiAXpVTrmsf/ZZYFfLczzHiGrQ50qD9M+mXbGrvrSJWMD41Q1coO4H70IVhXtG5nQ9du003p5lIxNSPQUmh3HSNVS5eq8bn5st11pFLGh4blDfTK5RMA2tY8/mqszezKBjAnsq4hy63XIjHSK5ZQ8Z3NiY8W/l9b18D3zhzDIjSsQdFHgXb0YVhaV/3tnyoWHVb/WYfjPPVyu901jFSb6xN3z767/UW760i1YREaAKGqle3APegJX+5Yc01rx9svP25zWbY7w7O5/w1BjCGztzXx5tK3Y1+zu46hMGxCAyBUtfJN4ElgIkDorRffjNRuH7HrbrhVNHFUoNWMZ6RZW1i1r90f/+BNy7pSspWoiGSLyBQRuUhE5okWEJHpIjI/Fa+RjOE4qv5PoBwoBOqbX374qdHnfqHQkztmqs11pd3J0TUNvoA+R8dIj3hCqVXV8S9c+nBoUGNqIvIcoIA5wFXAccBzwEeAcUqpzwL/QK/GfwBYCzyIPtN7B1AvIq8opdI2sXFYtTTgnRPa7kYvRBwkEU80r3jwr4lwqMnm0tLu/MTLZn5Gmq2qjt97zh87HkziIXuUUucCG4AY8AXg08CrwE3WfdzokDgZ+CzgR392S63rTk9F7YM17EIDIFS1ci9wJ3qKuTcRau5sXfX3h1Q8Fra5tLQ63bvFHDVJo7X74ytuXRH5SpIPKxCRRegAUMAjSqkblFJPAt8SkSsBl1LqQiACXA6MUUqdhd6B8KNKqeUp+hEGZViGBkCoauV64C/AJEAitdvr2994/m9qhJwO60mEE9P87WY8I02qGuLbb38tctmSLdFkJxaG0F3qevTUgdkiskxElgGz0Ovi3iki1wK7gG8C60TkMvQ+x0el7IcYpOE4ptHTv9FzN04FdnVWvVblySteFig94Tyb6xpyFdHX67xBfTKfMbRq2hP1D6yPXrx4Q+RwVpGLKKXWi0g7kI0+n6pbSCn1AoCIHADeAmZat52AHrcrR3dl0mbYtjTgnVPo/4geMNJLBK5Z8kqkfvcGWwtLg/PVK5121zAStIZV6NFN0Y/9+KXwlsN8iplWq2I2cAowr8dt/93j3/uBR3pdXgJScoQmGcM6NABCVSu7gN+gFyQuAGhZ8eDjsZYDb9ta2BA7zfu2Gc8YYl0xFX3szegNX3+q67A2OxIRN9DRYyA0AeSj/8CV8O7P53jg6l6XeYD38H+CwzPsQwPe2Q/2V+jNlgIqHo03LX/gL7GW2iqbSxsS3kQ4PjXQYQ61DqFwTEX/tCH6o39ujt17BE9zKbBdRM4ExlnXlQMXWpcAgIi4gCuBGuCr1iDoHcCLwGtH8PqHZUSEBkCoauUO4LfoBA+oWCTetPz3j8Za6zJm493BOi36Wr3HNezHq2wTjqnofWsjv3387diPl2yJHsnAegLdzfgosAndavipUupcq/VRb93vB+hxuW8rpSoBlFJ/B34PfF1E0joYKiPkYMI7gmUVJwNfQfcRu8Sb5Rl11nVXe/KKpttcWsr8qOu2nZ8o2Fhqdx3DUSSuove8Hrn/2e3xG5dsiaZ03EhEsgCUUo6eGjBiWhrdQlUrV3OwxeFX0XCsafnvH4m11m+3ubSUOdW31azSNQSswPj9UAQG6LBwemDACAwNgFDVypXoWaPjOBgcD8fa6jP+dPqsRChW6jfjGakWiavYfWuiDzy7PX7DUARGJhmRoQEQqlr5GgeDI0tFu2JN//n9Q5k+OHpG9NV6t0vMTmop1B5Rnbe/Frnn6W2xby7ZEh3x+weP2NAACFWtfBV9Ov14rOBofO6eh8M1VWkfkU6V89RKxzdvM0l9KNH8g+XhO1/aHb/JBIY2okMDIFS18hXgPnRw5KCUann54adDW1c+oVQi7RNnjlSFb5sZz0iRnc2Jmu8sC/9sS0Ni0ZItUXPyn2XEHT3pT7Cs4ljgq0AUaAAITJs7Naf8vKvE482ID6I/EYq94f+MmO7JkVu3P7791hXhH3fF+OOSLVGzn04PI76l0S1UtfIN4IfomaPjADq3v76j+ZWH70+EOxptLW6Q5kdfrjOBcWSUUjyzLbbhluXhr3fFeMAExnuZ0OghVLWyGh0cO4EpgETrdjY0Pv+7+2NtDTvtrG0wzlWvmfGMI9AZVZ13rIw885tVkS8u2RJ9YsmWaMZ1T9PBdE/6ECyr8AGfAs4E9gBR3B5X/ilXnO0bN/N0ERF7K+zbi4nraicHu4rtriMT7W5J7LttRXjZ3lZ165It0c121+NkJjT6ESyrcKHn/1+NnvMfAvBPmT055/jzL3f5AgV21tdbMN4WrQxc73a7xLQek5BQSv1nR3zjnasj/4ol+O2SLdEDdtfkdCY0BhAsqzgRuB7dldsP4PLn+PIqrrzIVzj5BFuL6+Girqf23VXwx/F215FJOqMqdNfrkZeX74z/AXhsyZZoxO6aMoEJjUEIllWMAa5Dn4G4F73sGsFZ847Onnn6B8XjC9pZH8Avwj/YeUX+llK768gUm2rjVbe/Fnn1QIe6HVh3hCeejSgmNAYpWFbhBs4CrgG60Muw4c4rysk/5cMf8uQXp33ZtZ5WqGvrJgbCZvr4ANrCquUP6yOrnt0efxm4a8mWaK3dNWUaExpJCpZVTAQ+j157dC96BWlyys89wT91zjkub1ZOumvKjTeHNwS/6HM5dIDWCRJKqVXV8Y23vxap7IjyT+Bx0x05PCY0DoN1dOUS9CIqTUAzgGRl+/JO+sA8X8mMU8XlStt8iUvDT+y7I/8hM57Rj4ZQova3qyOrV+9LrAF+v2RLdJfdNWUyExpHIFhWMQO9D0Ux+ghLF4B39MSCnBMuOt87atzR6ajj9vAtuz6UXzUlHa+VSUJR1bb07di6hyqjW+OKR4HnD2O1cKMXExpHyGp1zEPviOVF74QVB/CXnjgl+5j5F7oDeSVDWcMr6tP14wMRs12BJRJX4Rd3xVfdtyayuzPGeuBP5lBq6pjQSJFgWUU+ustyDtAJ6AE2EckpP+8E/5TZ812+QH6qXzc/3ti1LvjlLDOeAfGEiq/ZH1971+ro1oZOdQB4GHjdzOxMLRMaKRYsq5iEnhB2HPrEt1YAXG5X9tHzj/VPOf50dyAvZfuRXN71r+pfFjw6IVXPl4liCRXbVJvY+Lu1ka27WlQj8FdghRnoHBomNIZAsKxC0HM6PgkUAY10hwcQmH7K9MD0uad5cgunHelr3Rn+3q4P5G8bkeMZnVHVsao6vnrxhuje+pDqAp4AlpnT2IeWCY0hFCyr8AInApeh1+to5+AK02SNn1USnHnaaZ5R448VcR3W9O/XWNBQ4o+OSUnBGaKxU9X+Z0ds9SObonXhOAr4D/DEki3REbfJtx1MaKSBdR7L0egxj1nooyy1WLtjeUZNyM+eefocb9GU8mTOaRkVa+hcm/2VwEgYzognVHxns3r76W2xN57aGmtHL2HwFPDiki3RjFi6YLgwoZFGVrelFLgAqEAfZanBmiAG4J98/CT/5OPLvWMmHjvQ9PQru/6x92cFf504hCXbrrYjUf36vviGx96M7a0PKR96nGgJsHqkL/BrFxMaNgmWVRQD77cuHvQRlwa69+Z0uV2BaXOn+SceW+4pKJklbo+v93PcHb5514X5O4bdeEZjp6rdeCC+aenbsaotDQkv+mTBKmApsGnJlmjc3gpHNhMaNguWVfjRXZYz0ZsAu9CDps2AAhBfwBuYfnKZr3jqUZ68sdNdPn8ewCo+1Vjsj422qfSUicRVeE+L2v5mXXzb8p3xnVWNCR96zksb8BywGqgxJ5U5gwkNBwmWVeQCx6JPjJthXd1MjyMvHmKBWfnRK2XqqZsfHPfQ1Il5MsXvEdvPsk1GQqlEQ0jVVDUmtq6ujm97cVe8PpqgAN3iCgOvAKuAraZV4TwmNBwqWFYxGjgBHSDd4xbRAtpGl0n1+HwJPdt93xNKXIVzxrknTc53lYzNkeLRARnr90jAhrLfI6FUoqlT1dW0q327WhL736xL7FuzL17fESUHyEG3ppqB19E7p1eZ+RXOZkIjA1izTacBxxXTdNFM2ZOTJbF69LoeLejxkHeZnC855cXu4mmjXGMn5ElxfpbkZ/skN+gl1+fWe4amSkKpREeE1raIam7uUi2Nnaq5rkM1b2tK1L2+L34gFMUN5AJBdEjEgTfR3Y5twAHT9cgcJjQyzKUzvYKeMDYZOAY987QQa/zD+tqBnhPS50rauT68k/JdOeNzJbco6MrNzcLvcYnH68LtduE++FU8Ikg4piLhOJHOqIp0xYiEoiocihJpi6hwdatq29GcaI0lUOjxmCwgm4MBIejuVRWwBdgN7DSticxlQmMYuHSm148+07YYHSbT0Id2g3QfjTm48nwUPW4Qsf6trPv0/NqTu4+LBx0O3h6PEetrAzoYNgPV6BP42kxLYvgwoTFMWS2SXCAfPXbQ/e8iYAy6dZKDDoCel94zUyPo7k+X9bUTvchyHXrN1Gbr0oIOB3Ny2DBnQsN4l0tnel3o1oQLiJoQMHozoWEYRlLMHhmGYSTFhIZhGEkxoWEYRlJMaBiGkRQTGoZhJMWEhmEYSTGhYRhGUkxoGIaRFBMahmEkxYSGYRhJMaFhGEZSTGgYhpEUExqGYSTFhIZhGEkxoWEYRlJMaBiGkRQTGoZhJMWEhmEYSTGhYRhGUkxoGIaRFBMahmEkxYSGYRhJMaFhGEZSTGgYhpEUExqGYSTFhIZhGEkxoWEYRlJMaBiGkRQTGoZhJMWEhmEYSTGhYRhGUv4fYaJnrZj7YDIAAAAASUVORK5CYII=\n",
                            "text/plain": "<Figure size 432x288 with 1 Axes>"
                        },
                        "metadata": {}
                    }
                ],
                "layout": "IPY_MODEL_12810ce0ba8e479183f1a775fb1e9abb"
            }
        }
    }
}
</script>
</head>
<body>

<script type="application/vnd.jupyter.widget-view+json">
{
    "version_minor": 0,
    "model_id": "7971196dea494f95ac9c502564cdf669",
    "version_major": 2
}
</script>

<script type="application/vnd.jupyter.widget-view+json">
{
    "version_minor": 0,
    "model_id": "4b352cf198fc46cb92952aa63b68e8e8",
    "version_major": 2
}
</script>

</body>
</html>


---
顯示結果：
![互動式輸入](MidProj/MidProj1.png)
![統計表圓餅圖](MidProj/MidProj2.png)